# get strided patch array

In [16]:
import time
nb_start_time = time.time()

import os
import tempfile
import sys

import numpy as np
import pandas as pd

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import get_level_sizes_dict, get_sample_selection_mask

#                                              Notebook conveniences
def check_patch_in_bounds(x, y, X_dim, Y_dim):
    """ Usage: TrueFalse = check_patch_in_bounds(x, y, X_dim, Y_dim)
                determine if the box is within the image
    Args:
        x, y:           a tuple, list or array (x_start, x_end) (y_start, Y_end)
        X_dim, Y_dim:   a tuple, list or array (Image_X_start, Image_X_end), (Y - etc.)
    """
    if x[0] > x[1] or y[0] > y[1] or X_dim[0] > X_dim[1] or Y_dim[0] > Y_dim[1]:
        return False
    if x[0] >= X_dim[0] and y[0] >= Y_dim[0] and x[1] < X_dim[1] and y[1] < Y_dim[1]:
        return True
    else:
        return False
    
def im_pair_hori(im_0, im_1):
    """ Usage: new_im = im_pair_hori(im_0, im_1)
            combine a list of PIL images horizontaly
    """
    w0 = im_0.size[0]
    w = w0 + im_1.size[0] + 1
    h = max(im_0.size[1], im_1.size[1])

    new_im = tip.Image.new('RGB', (w, h) )
    box = (0, 0, w0, h)
    new_im.paste(im_0, box)
    
    box = (w0+1, 0, w, h)
    new_im.paste(im_1, box)

    return new_im

In [17]:
test_data_dir = '../../DigiPath_MLTK_data/RegistrationDevData/'
os.listdir(test_data_dir)

offset_data_file = os.path.join(test_data_dir, 'wsi_pair_sample.csv')
if os.path.isfile(offset_data_file):
    offset_df = pd.read_csv(offset_data_file)

print('wsi_pair_sample.csv:\n',offset_df.iloc[0], '\n')

offset_x = offset_df['truth_offset_x'].iloc[0]
offset_y = offset_df['truth_offset_y'].iloc[0]
offset_x, offset_y = int(round(offset_x)), int(round(offset_y))

auto_x = offset_df['auto_offset_x'].iloc[0]
auto_y = offset_df['auto_offset_y'].iloc[0]
auto_x, auto_y = int(round(auto_x)), int(round(auto_y))

# print('\noffset_x, offset_y, auto_x, auto_y\n', offset_x, offset_y, auto_x, auto_y)

fixed_wsi = os.path.join(test_data_dir, '54742d6c5d704efa8f0814456453573a.tiff')
fixed_levels_dict = get_level_sizes_dict(fixed_wsi)
fixed_max_width = fixed_levels_dict['image_size'][0]
fixed_max_height = fixed_levels_dict['image_size'][1]
    
float_wsi = os.path.join(test_data_dir, 'e39a8d60a56844d695e9579bce8f0335.tiff')
float_levels_dict = get_level_sizes_dict(float_wsi)
float_max_width = float_levels_dict['image_size'][0]
float_max_height = float_levels_dict['image_size'][1]

    
run_parameters = {'wsi_filename': fixed_wsi, 
                  'wsi_floatname': float_wsi,
                  'thumbnail_divisor': 20, 
                  'patch_select_method': 'threshold_rgb2lab', 
                  'rgb2lab_threshold': 80, 
                  'image_level': 0, 
                  'patch_height': 224, 
                  'patch_width': 224, 
                  'threshold': 0, 
                  'offset_x': offset_x, 
                  'offset_y': offset_y}

for k, v in run_parameters.items():
    print('%25s: %s'%(k,v))
    
print('\n\nfixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff')
for k, v in fixed_levels_dict.items():
    print('%25s: %s'%(k,v))

print('\n\nfloat_wsi image: e39a8d60a56844d695e9579bce8f0335.tiff')
for k, v in float_levels_dict.items():
    print('%25s: %s'%(k,v))

wsi_pair_sample.csv:
 fixed_wsi         54742d6c5d704efa8f0814456453573a
float_wsi         e39a8d60a56844d695e9579bce8f0335
truth_offset_x                            -1617.86
truth_offset_y                             1672.74
auto_offset_x                             -1620.95
auto_offset_y                               1675.6
Name: 0, dtype: object 

             wsi_filename: ../../DigiPath_MLTK_data/RegistrationDevData/54742d6c5d704efa8f0814456453573a.tiff
            wsi_floatname: ../../DigiPath_MLTK_data/RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff
        thumbnail_divisor: 20
      patch_select_method: threshold_rgb2lab
        rgb2lab_threshold: 80
              image_level: 0
             patch_height: 224
              patch_width: 224
                threshold: 0
                 offset_x: -1618
                 offset_y: 1673


fixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff
               image_size: (145408, 83968)
              level_count: 10
       

In [18]:
def get_offset_array_start_and_end(array_length, array_offset):
    """ Usage: array_start, array_end = get_offset_array_start_end(array_length, array_offset)
    
    Args:
        array_length:   one image dimension
        array_offset:   registration offset
        
    Returns:
        array_start:    first index in image diminsion
        array_end:      last index + 1 in image diminsion
    """
    array_start = max(0, array_offset)
    array_end = min(array_length, array_length + array_offset)
    
    return array_start, array_end

image_size = [0, 40985]
print('image_size', image_size, '\n')
array_length = image_size[1]
array_offset = -1698

array_start, array_end = get_offset_array_start_and_end(array_length, array_offset)
print('Offset: %i, Array Start: %i, Array End: %i'%(array_offset, array_start, array_end), '\n')

array_offset = -array_offset
array_start, array_end = get_offset_array_start_and_end(array_length, array_offset)
print('Offset: %i, Array Start: %i, Array End: %i'%(array_offset, array_start, array_end), '\n')

image_size [0, 40985] 

Offset: -1698, Array Start: 0, Array End: 39287 

Offset: 1698, Array Start: 1698, Array End: 40985 



In [27]:
def get_strided_fence_array(patch_len, patch_stride, arr_start, arr_end):
    """ Usage: sf_array = get_strided_fence_array(patch_len, patch_stride, arr_start, arr_end)
    
    Args:
        patch_len:      patch dimension
        patch_stride:   percent of patch dimension - stride = floor(patch_len * patch_stride)
        arr_start:      beginning index using closed zero-based indexing
        arr_end:        last index using closed zero-based indexing
        
    Returns:
        fence_array:    numpy array [[p_start, p_end], [p_start, p_end],...]
        
    """
    fence_stride = int(patch_len * patch_stride)
    array_size = 1 + (arr_end - arr_start) // fence_stride
    fence_array = np.zeros((array_size, 2))
    
    p_loc = np.array([arr_start, arr_start+patch_len])
    pair_number = 0
    while p_loc[1] < arr_end:
        fence_array[pair_number, :] = p_loc
        pair_number += 1
        p_loc[0] = p_loc[0] + fence_stride
        p_loc[1] = p_loc[0] + patch_len
        
    if pair_number < fence_array.shape[0]:
        if fence_array[pair_number, 1] != arr_end - 1:
            fence_array[pair_number, :] = (arr_end - patch_len - 1, arr_end - 1)
            
        fence_array = fence_array[0:pair_number + 1, :]
        
    return fence_array

n_to_show = 4

patch_len = 224
patch_stride = 1.1

arr_start = 0
arr_end = 63042 + 1

fence_array = get_strided_fence_array(patch_len, patch_stride, arr_start, arr_end)
print('array start - end',arr_start, arr_end, '\tresult size:', fence_array.shape)

print('\nFrom the Begining Forward')
for fp in range(0, min(n_to_show, fence_array.shape[0])):
    print(fence_array[fp, :], '\t', fence_array[fp, 1] - fence_array[fp, 0])
    
print('\nFrom the Ending Backwards')
for fp in range(fence_array.shape[0]-1, fence_array.shape[0] - n_to_show - 1, - 1):
    print(fp, fence_array[fp, :], '\t', fence_array[fp, 1] - fence_array[fp, 0])

array start - end 0 63043 	result size: (257, 2)

From the Begining Forward
[  0. 224.] 	 224.0
[246. 470.] 	 224.0
[492. 716.] 	 224.0
[738. 962.] 	 224.0

From the Ending Backwards
256 [62818. 63042.] 	 224.0
255 [62730. 62954.] 	 224.0
254 [62484. 62708.] 	 224.0
253 [62238. 62462.] 	 224.0


In [ ]:
def get_registration_patch_location_arrays():
    pass